In [1]:
import logging
import sys
import os

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)

In [2]:
from llama_index.llms import Ollama
from llama_index import ServiceContext
from llama_index import KnowledgeGraphIndex
from llama_index.retrievers import (KnowledgeGraphRAGRetriever, KGTableRetriever)
from llama_index.graph_stores import Neo4jGraphStore
from llama_index.storage.storage_context import StorageContext
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.data_structs.data_structs import KG
from IPython.display import Markdown, display

In [3]:
llm = Ollama(model='mistral', base_url="http://localhost:11434")
service_context = ServiceContext.from_defaults(llm=llm, embed_model="local:BAAI/bge-small-en")

In [4]:
graph_store = Neo4jGraphStore(username="neo4j", password="password", url="bolt://localhost:7687", database="neo4j")
storage_context = StorageContext.from_defaults(graph_store=graph_store)

In [5]:
kg_index = KnowledgeGraphIndex(index_struct=KG(index_id="vector"), service_context=service_context, storage_context=storage_context)
graph_rag_retriever = KGTableRetriever(index=kg_index, retriever_mode="keyword")

In [6]:
nodes = graph_rag_retriever.retrieve("Tell me about Peter Quill.")
len(nodes)

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


1

In [7]:
kg_rag_query_engine = RetrieverQueryEngine.from_args(retriever=graph_rag_retriever, service_context=service_context)

In [8]:
response_graph_rag = kg_rag_query_engine.query("Tell me about Peter Quill.")
display(Markdown(f"<b>{response_graph_rag}</b>"))

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


<b> Peter Quill is an entity in the given schema with various relationships to other entities. He is the leader of the Guardians and was first physically released on a specific date. He has acted in several films, some of which were released before others. He portrays certain characters and provides voice and motion capture for them. He had narrowed down choices for potential songs to use in his films and made tweets regarding them. He was expected to be paid for his work and called for the firing of someone, stating reasons for it. He has been involved with various other entities as well, such as developing psionic abilities after being abandoned and changing the gender of a cosmic entity. However, without further context, it is unclear what specific aspects of Peter Quill's character you are interested in.</b>

## new api (broken)

In [7]:
# graph_rag_retriever = KnowledgeGraphRAGRetriever(service_context=service_context, storage_context=storage_context, retriever_mode="keyword_embedding")